In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm

In [2]:

Apath_to_df_ada="v2_lagging_A22apr_22may_2021_ada.hdf"
Apath_to_df_ada="v2_lagging_C28may_28may_2020_2021_ada.hdf"
Cpath_to_df_ada="v2_lagging_C28may_28may_2020_2021_ada.hdf"
#Apath_to_df_ada = "v1_28may_28may_2020_2021_ada.hdf"

Apath_to_df_eth="v2_lagging_A28may_28may_2020_2021_eth.hdf"


df = pd.read_hdf(Apath_to_df_ada)
df = df[:-180]
a=len(df)-1
df.iloc[a : a + 1]

,trade_in_3h_usdtfutures_ADAUSDT_close_price,trade_in_2h_usdtfutures_ADAUSDT_close_price,trade_in_1h_usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_volume,SMA(usdtfutures_ADAUSDT_close_price_5),SMA(usdtfutures_ADAUSDT_volume_5),SMA(usdtfutures_ADAUSDT_close_price_60),shift1_SMA(usdtfutures_ADAUSDT_close_price_60),shift2_SMA(usdtfutures_ADAUSDT_close_price_60),...,shift8_SMA(usdtfutures_ADAUSDT_close_price_240),SMA(usdtfutures_ADAUSDT_volume_240),shift1_SMA(usdtfutures_ADAUSDT_volume_240),shift2_SMA(usdtfutures_ADAUSDT_volume_240),shift3_SMA(usdtfutures_ADAUSDT_volume_240),shift4_SMA(usdtfutures_ADAUSDT_volume_240),shift5_SMA(usdtfutures_ADAUSDT_volume_240),shift6_SMA(usdtfutures_ADAUSDT_volume_240),shift7_SMA(usdtfutures_ADAUSDT_volume_240),shift8_SMA(usdtfutures_ADAUSDT_volume_240)
2021-06-10 04:36:59.999000+00:00,-0.012488,-0.006875,-0.006433,-0.001197,-0.654268,-0.000691,0.144105,-0.000182,0.000062,-0.000113,...,0.000194,0.256596,0.228429,0.240134,0.237042,0.202097,0.220119,0.154016,0.18795,0.209585


In [3]:
def check_x_for_leak(X):
    for column in X.columns:
        assert "trade_in_" not in column

    

In [4]:
def create_model(df_orig, index, train_size, test_size, coin):
    df_orig = df_orig.fillna(0)
    del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    start = index - (train_size+test_size)
    end = index
    df_orig = df_orig[start:end]

    df = df_orig.copy()
    df = df.sample(frac=1/4)
    Y = df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)]

    Y_MEAN = np.mean(np.abs(Y))
    Y_MEAN = Y_MEAN * (1/2)

    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] > Y_MEAN] = 20
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < -Y_MEAN] = 10
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < 9] = 1

    first_element = "usdtfutures_{}USDT_close_price".format(coin)
    #first_element = "usdtfutures_{}USDT_close_price_ewm_mean".format(coin)
    
    X = df.loc[:, first_element :]
    check_x_for_leak(X)

    test_size = test_size / (test_size+train_size)
    test_size = test_size or 0.01
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=6, shuffle=False
    )
    hours_in_test = len(Y_test)

    model = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        #booster=["gbtree","gblinear"][0],
        gamma=0.1,
        #sampling_method=["uniform", "gradient_based"][1],
        #num_parallel_tree=1,
        #reg_alpha=1,
        #reg_lambda=0,
        #eta=0.3,
    )
    model.fit(X_train, Y_train)
    return model
    

In [5]:
def predict_row(df_orig, index, model, coin):
    #del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    #del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    df_orig = df_orig.iloc[index:index+1]
    df_orig = df_orig.copy()
    df_orig = df_orig.fillna(0)
    
    first_element = "usdtfutures_{}USDT_close_price".format(coin)
    #first_element = "usdtfutures_{}USDT_close_price_ewm_mean".format(coin)
    X = df_orig.loc[:,first_element :]
    check_x_for_leak(X)
    y_pred = model.predict(X)[0]
    Y_TEST_REAL = df_orig["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)][0]
    
    profit = -1
    
    side = -1
    if y_pred == 20:
        side = 1
    if y_pred == 1:
        return Y_TEST_REAL, 0

    if side * Y_TEST_REAL > 0:
        profit = 1
    
    return abs(Y_TEST_REAL) * profit, side
    
    

    
    

In [ ]:
predictsA = []
predictsB = []
predictsC = []
model_A = None
model_B = None
model_C = None
last_model_train_A = 0
last_model_train_B = 0
last_model_train_C = 0
retrain_limit_A = 60*24*6
retrain_limit_B = 60*24*6
retrain_limit_C = 60*24*6
df_orig_A = pd.read_hdf("v2_lagging_C28may_28may_2020_2021_ada.hdf")
df_orig_B = pd.read_hdf("v2_lagging_B28may_28may_2020_2021_ada.hdf")
df_orig_C = pd.read_hdf("v2_lagging_C28may_28may_2020_2021_ada.hdf")
for i in tqdm.tqdm(range(len(df_orig_C) - (60*24*30), len(df_orig_C)-2, 60)):

    if (i - last_model_train_A) > retrain_limit_A:
        model_A = None
    if (i - last_model_train_B) > retrain_limit_B:
        model_B = None
    if (i - last_model_train_C) > retrain_limit_C:
        model_C = None

    

    if model_A is None:
        #model_A = create_model(df_orig_A.copy(), i-61, (60*24*20), 0, "ADA")
        last_model_train_A = i
    #predictA = predict_row(df_orig_A, i, model_A, "ADA")

    
    if model_B is None:
        #model_B = create_model(df_orig_B.copy(), i-61, (60*24*30), 0, "ADA")
        last_model_train_B = i
    #predictB = predict_row(df_orig_B, i, model_B, "ADA")



    if model_C is None:
        model_C = create_model(df_orig_C.copy(), i-61, (60*24*60), 0, "ADA")
        last_model_train_C = i
    predictC = predict_row(df_orig_C, i, model_C, "ADA")
    
    

    predictsA.append(predictC)
    predictsB.append(predictC)
    predictsC.append(predictC)


  0%|          | 0/720 [00:00<?, ?it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:15:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 19%|█▉        | 139/720 [08:46<17:26,  1.80s/it]   /home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:23:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 40%|████      | 289/720 [09:13<02:23,  3.01it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:24:17] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 59%|█████▊    | 422/720 [09:35<00:40,  7.32it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:24:40] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
initial_bank = 100
rake = 0.00018

last_side = -1
last_coin = "ADA"
coin="ADA"
for i in range(len(predictsA)):
    if i ==0:
        continue

    predicts = predictsA
    
    win_len = 50
    A_sum = predictsA[i-win_len:i-1]
    B_sum = predictsB[i-win_len:i-1]
    C_sum = predictsC[i-win_len:i-1]
    A_sum  = sum([x[0] for x in A_sum if x[1] !=0])
    B_sum  = sum([x[0] for x in B_sum if x[1] !=0])
    C_sum  = sum([x[0] for x in C_sum if x[1] !=0])
    
    
    predicts= predictsA
    if A_sum < B_sum:
        predicts = predictsB
        if B_sum < C_sum:
            predicts = predictsC
    predicts= predictsC

    
    p, side = predicts[i]
    if side ==0:
        profit = -1
        if p * last_side > 0:
            profit = 1
        p = abs(p)*profit
        side = last_side


    change = initial_bank * p
    initial_bank = initial_bank + change
    
    if side != last_side or last_coin != coin:
        rake_change = initial_bank * rake * -2
        initial_bank = initial_bank + rake_change
    
    last_side = side
    last_coin = coin

    
    
    
    
initial_bank

In [ ]:
# A always rake

initial_bank = 100
rake = 0.00018


for i in range(len(predictsA)):
    if i ==0:
        continue

    predicts = predictsA
    p, side = predicts[i]
    if side ==0:
        rake_change = initial_bank * rake * -2
        initial_bank = initial_bank + rake_change
        continue



    change = initial_bank * p
    initial_bank = initial_bank + change
    
    rake_change = initial_bank * rake * -2
    initial_bank = initial_bank + rake_change

    
initial_bank

In [ ]:
# B always rake

initial_bank = 100
rake = 0.00018


for i in range(len(predictsB)):
    if i ==0:
        continue

    predicts = predictsB
    p, side = predicts[i]
    if side ==0:
        rake_change = initial_bank * rake * -2
        initial_bank = initial_bank + rake_change
        continue



    change = initial_bank * p
    initial_bank = initial_bank + change
    
    rake_change = initial_bank * rake * -2
    initial_bank = initial_bank + rake_change

    
initial_bank

In [ ]:
# C always rake

initial_bank = 100
rake = 0.00018


for i in range(len(predictsC)):
    if i ==0:
        continue

    predicts = predictsC
    p, side = predicts[i]
    if side ==0:
        rake_change = initial_bank * rake * -2
        initial_bank = initial_bank + rake_change
        continue



    change = initial_bank * p
    initial_bank = initial_bank + change
    
    rake_change = initial_bank * rake * -2
    initial_bank = initial_bank + rake_change

    
initial_bank